# Clinical Trial Adverse Events Data Agent
## Three Example Queries with LLM Testing

This notebook demonstrates the ClinicalTrialDataAgent with three practical queries about adverse events data, plus API key testing.

## Setup: Test API Key First

Make sure you upload your API key to .env

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if API key is set
api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print(f"✓ API Key found: {api_key[:20]}...{api_key[-10:]}")
else:
    print("✗ No API key found in .env file")

✓ API Key found: sk-proj-fAJhTgAuzXhC...RGmaACDHkA


In [2]:
# Test the API key with a simple call
from langchain_openai import ChatOpenAI

print("Testing OpenAI API connection...")
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        api_key=os.getenv("OPENAI_API_KEY"),
        timeout=10
    )
    
    # Send a quick test message
    response = llm.invoke("Say 'OK' if you can read this")
    print(f"\n✓ API KEY IS WORKING!")
    print(f"Response: {response.content}")
    api_working = True
except Exception as e:
    error_msg = str(e)
    print(f"\n✗ API Connection Failed")
    print(f"Error: {error_msg}")
    print("\nNote: The agent will use fallback pattern matching instead")
    api_working = False

Testing OpenAI API connection...

✓ API KEY IS WORKING!
Response: OK


## Load Data and Initialize Agent

In [3]:
import pandas as pd
from clin_assistant import ClinicalTrialDataAgent

# Load the adverse events data
AE_DATA = pd.read_csv('ae.csv')
print(f"✓ Loaded adverse events dataset")
print(f"  - Records: {AE_DATA.shape[0]}")
print(f"  - Columns: {AE_DATA.shape[1]}")
print(f"\nColumns: {list(AE_DATA.columns)}")

# Initialize the agent
agent = ClinicalTrialDataAgent()
print(f"\n✓ ClinicalTrialDataAgent initialized")

✓ Loaded adverse events dataset
  - Records: 1191
  - Columns: 35

Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'AESEQ', 'AESPID', 'AETERM', 'AELLT', 'AELLTCD', 'AEDECOD', 'AEPTCD', 'AEHLT', 'AEHLTCD', 'AEHLGT', 'AEHLGTCD', 'AEBODSYS', 'AEBDSYCD', 'AESOC', 'AESOCCD', 'AESEV', 'AESER', 'AEACN', 'AEREL', 'AEOUT', 'AESCAN', 'AESCONG', 'AESDISAB', 'AESDTH', 'AESHOSP', 'AESLIFE', 'AESOD', 'AEDTC', 'AESTDTC', 'AEENDTC', 'AESTDY', 'AEENDY']

✓ ClinicalTrialDataAgent initialized


## Example 1: Find subjects with Moderate Severity Adverse Events

In [5]:
query_1 = "Give me the subjects who had Adverse events of Moderate severity"
print(f"Query: {query_1}\n")

result_1 = agent.query(query_1)

if result_1['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_1['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_1['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_1['filter_applied']}")
    print(f"  Matching Records: {result_1['matching_records']}")
    print(f"  Unique Subjects: {result_1['unique_subjects']}")
else:
    print(f"✗ Error: {result_1.get('error')}")

Query: Give me the subjects who had Adverse events of Moderate severity

✓ Query executed successfully

Parsed Intent:
  Target Column: AESEV
  Filter Value: MODERATE

Results:
  Filter Applied: AESEV = MODERATE
  Matching Records: 378
  Unique Subjects: 136


In [6]:
# Display the filtered dataframe
print(f"Filtered Adverse Events Data - Moderate Severity ({len(result_1['subject_ids'])} unique subjects):")
print()

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_1['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_1['parsed_intent']['filter_value'].upper())
]

print(filtered_df.to_string(index=False))
print(f"\nTotal Records: {len(filtered_df)}")
print(f"Unique Subjects: {len(filtered_df['USUBJID'].unique())}")

Filtered Adverse Events Data - Moderate Severity (136 unique subjects):

     STUDYID DOMAIN     USUBJID  AESEQ AESPID                                 AETERM                                AELLT  AELLTCD                                AEDECOD  AEPTCD    AEHLT  AEHLTCD    AEHLGT  AEHLGTCD                                             AEBODSYS  AEBDSYCD                                                AESOC  AESOCCD    AESEV AESER  AEACN    AEREL                      AEOUT AESCAN AESCONG AESDISAB AESDTH AESHOSP AESLIFE AESOD      AEDTC    AESTDTC    AEENDTC  AESTDY  AEENDY
CDISCPILOT01     AE 01-701-1023      2    E09                               ERYTHEMA                   LOCALIZED ERYTHEMA      NaN                               ERYTHEMA     NaN HLT_0284      NaN HLGT_0192       NaN               SKIN AND SUBCUTANEOUS TISSUE DISORDERS       NaN               SKIN AND SUBCUTANEOUS TISSUE DISORDERS      NaN MODERATE     N    NaN PROBABLE NOT RECOVERED/NOT RESOLVED      N       N        N    

## Example 2: Find subjects with Cardiac Events

In [7]:
query_2 = "Which subjects experienced cardiac events?"
print(f"Query: {query_2}\n")

result_2 = agent.query(query_2)

if result_2['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_2['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_2['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_2['filter_applied']}")
    print(f"  Matching Records: {result_2['matching_records']}")
    print(f"  Unique Subjects: {result_2['unique_subjects']}")
else:
    print(f"✗ Error: {result_2.get('error')}")

Query: Which subjects experienced cardiac events?

✓ Query executed successfully

Parsed Intent:
  Target Column: AESOC
  Filter Value: CARDIAC DISORDERS

Results:
  Filter Applied: AESOC = CARDIAC DISORDERS
  Matching Records: 91
  Unique Subjects: 44


In [8]:
# Display the filtered dataframe
print(f"Filtered Adverse Events Data - Cardiac Events ({len(result_2['subject_ids'])} unique subjects):")
print()

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_2['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_2['parsed_intent']['filter_value'].upper())
]

print(filtered_df.to_string(index=False))
print(f"\nTotal Records: {len(filtered_df)}")
print(f"Unique Subjects: {len(filtered_df['USUBJID'].unique())}")

Filtered Adverse Events Data - Cardiac Events (44 unique subjects):

     STUDYID DOMAIN     USUBJID  AESEQ AESPID                               AETERM                                AELLT  AELLTCD                              AEDECOD  AEPTCD    AEHLT  AEHLTCD    AEHLGT  AEHLGTCD          AEBODSYS  AEBDSYCD             AESOC  AESOCCD    AESEV AESER  AEACN    AEREL                      AEOUT AESCAN AESCONG AESDISAB AESDTH AESHOSP AESLIFE AESOD      AEDTC    AESTDTC    AEENDTC  AESTDY  AEENDY
CDISCPILOT01     AE 01-701-1023      3    E10 ATRIOVENTRICULAR BLOCK SECOND DEGREE               AV BLOCK SECOND DEGREE      NaN ATRIOVENTRICULAR BLOCK SECOND DEGREE     NaN HLT_0415      NaN HLGT_0086       NaN CARDIAC DISORDERS       NaN CARDIAC DISORDERS      NaN     MILD     N    NaN POSSIBLE NOT RECOVERED/NOT RESOLVED      N       N        N      N       N       N     N 2012-08-27 2012-08-26        NaN    22.0     NaN
CDISCPILOT01     AE 01-701-1047      4    E09             BUNDLE BRANCH BLOCK

## Example 3 (COMPLEX): Find me subjects that had a serious AE

In [13]:
query_3 = "Find me subjects that had a serious AE"
print(f"Query: {query_3}\n")

result_3 = agent.query(query_3)

if result_3['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_3['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_3['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_3['filter_applied']}")
    print(f"  Matching Records: {result_3['matching_records']}")
    print(f"  Unique Subjects: {result_3['unique_subjects']}")
else:
    print(f"✗ Error: {result_3.get('error')}")

Query: Find me subjects that had a serious AE

✓ Query executed successfully

Parsed Intent:
  Target Column: AESER
  Filter Value: Y

Results:
  Filter Applied: AESER = Y
  Matching Records: 3
  Unique Subjects: 3


In [14]:
# Display the filtered dataframe
print(f"Filtered Adverse Events Data - Recovered Events ({len(result_3['subject_ids'])} unique subjects):")
print()

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_3['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_3['parsed_intent']['filter_value'].upper())
]

print(filtered_df.to_string(index=False))
print(f"\nTotal Records: {len(filtered_df)}")
print(f"Unique Subjects: {len(filtered_df['USUBJID'].unique())}")

Filtered Adverse Events Data - Recovered Events (3 unique subjects):

     STUDYID DOMAIN     USUBJID  AESEQ AESPID                                         AETERM                                          AELLT  AELLTCD                                        AEDECOD  AEPTCD    AEHLT  AEHLTCD    AEHLGT  AEHLGTCD                 AEBODSYS  AEBDSYCD                    AESOC  AESOCCD    AESEV AESER  AEACN    AEREL              AEOUT AESCAN AESCONG AESDISAB AESDTH AESHOSP AESLIFE AESOD      AEDTC    AESTDTC    AEENDTC  AESTDY  AEENDY
CDISCPILOT01     AE 01-709-1424      1    E05                                        SYNCOPE                                        SYNCOPE      NaN                                        SYNCOPE     NaN HLT_0440      NaN HLGT_0310       NaN NERVOUS SYSTEM DISORDERS       NaN NERVOUS SYSTEM DISORDERS      NaN MODERATE     Y    NaN POSSIBLE RECOVERED/RESOLVED      N       N        N      N       N       Y     N 2013-03-08 2013-03-07 2013-03-07     5.0     5.0
CDIS

## Summary Comparison

In [15]:
# Create summary table
summary_data = [
    {
        'Query': 'Moderate Severity',
        'Filter': result_1['filter_applied'],
        'Records': result_1['matching_records'],
        'Unique Subjects': result_1['unique_subjects']
    },
    {
        'Query': 'Cardiac Events',
        'Filter': result_2['filter_applied'],
        'Records': result_2['matching_records'],
        'Unique Subjects': result_2['unique_subjects']
    },
    {
        'Query': 'Recovered Events',
        'Filter': result_3['filter_applied'],
        'Records': result_3['matching_records'],
        'Unique Subjects': result_3['unique_subjects']
    }
]

summary_df = pd.DataFrame(summary_data)
print("\nSummary of All Queries:")
print(summary_df.to_string(index=False))


Summary of All Queries:
            Query                    Filter  Records  Unique Subjects
Moderate Severity          AESEV = MODERATE      378              136
   Cardiac Events AESOC = CARDIAC DISORDERS       91               44
 Recovered Events                 AESER = Y        3                3


## Notes

- The agent can use either LLM-based parsing or fallback pattern matching
- Each query uses minimal tokens since it just converts natural language to structured filters
- The agent returns unique subject counts and complete subject IDs for further analysis

# Clinical Trial Adverse Events Data Agent
## Three Example Queries

This notebook demonstrates the ClinicalTrialDataAgent with three practical queries about adverse events data.

### Setup: Import and Initialize

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load the adverse events data
AE_DATA = pd.read_csv('ae.csv')
print(f"Loaded adverse events dataset: {AE_DATA.shape[0]} records, {AE_DATA.shape[1]} columns")
print(f"\nFirst few rows:")
print(AE_DATA.head())
print(f"\nColumns: {list(AE_DATA.columns)}")

In [ ]:
# Import the ClinicalTrialDataAgent
from clin_assistant import ClinicalTrialDataAgent

# Initialize the agent
agent = ClinicalTrialDataAgent()
print("✓ ClinicalTrialDataAgent initialized")

## Example 1: Find subjects with Moderate Severity Adverse Events

**Query:** "Give me the subjects who had Adverse events of Moderate severity"

In [ ]:
query_1 = "Give me the subjects who had Adverse events of Moderate severity"
print(f"Query: {query_1}\n")

result_1 = agent.query(query_1)

if result_1['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_1['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_1['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_1['filter_applied']}")
    print(f"  - Matching Records: {result_1['matching_records']}")
    print(f"  - Unique Subjects: {result_1['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_1['subject_ids'])}) :")
    for subj_id in result_1['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_1['subject_ids']) > 15:
        print(f"  ... and {len(result_1['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_1.get('error')}")

In [ ]:
# Show sample data from results
print("Sample Adverse Events (first 5):")
print()
for i, record in enumerate(result_1['sample_data'], 1):
    print(f"{i}. {record}")
    print()

## Example 2: Find subjects with Cardiac Events

**Query:** "Which subjects experienced cardiac events?"

In [ ]:
query_2 = "Which subjects experienced cardiac events?"
print(f"Query: {query_2}\n")

result_2 = agent.query(query_2)

if result_2['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_2['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_2['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_2['filter_applied']}")
    print(f"  - Matching Records: {result_2['matching_records']}")
    print(f"  - Unique Subjects: {result_2['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_2['subject_ids'])}) :")
    for subj_id in result_2['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_2['subject_ids']) > 15:
        print(f"  ... and {len(result_2['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_2.get('error')}")

In [ ]:
# Show sample data from results
print("Sample Cardiac Adverse Events (first 5):")
print()
for i, record in enumerate(result_2['sample_data'], 1):
    print(f"{i}. {record}")
    print()

## Example 3: Find subjects with Recovered Adverse Events

**Query:** "Show me all subjects with recovered adverse events"

In [ ]:
query_3 = "Show me all subjects with recovered adverse events"
print(f"Query: {query_3}\n")

result_3 = agent.query(query_3)

if result_3['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_3['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_3['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_3['filter_applied']}")
    print(f"  - Matching Records: {result_3['matching_records']}")
    print(f"  - Unique Subjects: {result_3['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_3['subject_ids'])}) :")
    for subj_id in result_3['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_3['subject_ids']) > 15:
        print(f"  ... and {len(result_3['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_3.get('error')}")

In [ ]:
# Show sample data from results
print("Sample Recovered Adverse Events (first 5):")
print()
for i, record in enumerate(result_3['sample_data'], 1):
    print(f"{i}. {record}")
    print()